In [1]:
import numpy as np
import h5py
import torch
#from torch.utils.data import Dataset, DataLoader
# import torch.utils.data as data
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

/Users/wilsonchang/Desktop/Project/2ndtest/.env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Helper Clases

In [3]:
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() 
        return x.view(N, -1)

# Load Data

In [36]:
N, C, H, W = 128, 3, 32, 32
X = torch.randn(N, C, H, W)
y = np.random.randint(0, 10, size=128)

# Make DataLoaders

Random Data to test if the model learn

In [35]:
X = torch.randn(N, E, T)
y = np.random.randint(0, 10, size=N)

NameError: name 'E' is not defined

In [37]:
bs = 50
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-100, 50, 50])
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=bs, shuffle=False, num_workers=4) for x in ['train', 'val', 'test']}

# Define Model

In [38]:
dtype = torch.FloatTensor
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=7, stride=1),
    nn.ReLU(inplace=True),
    nn.BatchNorm2d(num_features=32),
    nn.MaxPool2d(kernel_size=2, stride=2),
    Flatten(),
    nn.Linear(5408, 1024),
    nn.ReLU(inplace=True),
    nn.Linear(1024, 10),
)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training

In [43]:
num_epochs = 10
for epoch in range(num_epochs):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        X_train, y_train = Variable(X_train), Variable(y_train)
        X_train.type(dtype)
        y_train.type(dtype)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(X_train)
        loss = loss_fn(outputs, y_train.long())
        print('train loss: %f' % (loss))
        loss.backward()
        # Update the parameters
        optimizer.step()
        # print Grad
        ## Validation Accuracy After 1 epoch    
        y_pred_tot = []
        y_val_tot = []
        for i_batch, sample_batched in enumerate(dataloaders['val']):
            X_val, y_val = sample_batched
            X_val, y_val = Variable(X_val), Variable(y_val)
            # Calculate the loss
            scores = model(X_val)
            loss = loss_fn(scores, y_val.long())
            print('val loss: %f' % loss)
            _, y_pred = torch.max(scores, 1)
            # print (y_pred.data.numpy() == y_val.data.numpy())
            y_pred_tot.append(y_pred.data.numpy())
            y_val_tot.append(y_val.data.numpy())
        
        val_acc = np.mean(np.concatenate(y_pred_tot) == np.concatenate(y_val_tot))
        print('(Epoch %d / %d) val_acc: %f' % (epoch+1, num_epochs, val_acc))

train loss: 0.003194
val loss: 7.620437
(Epoch 1 / 10) val_acc: 0.080000
train loss: 0.004782
val loss: 7.784849
(Epoch 2 / 10) val_acc: 0.080000
train loss: 0.006172
val loss: 7.836051
(Epoch 3 / 10) val_acc: 0.080000
train loss: 0.006625
val loss: 7.780218
(Epoch 4 / 10) val_acc: 0.080000
train loss: 0.005831
val loss: 7.642165
(Epoch 5 / 10) val_acc: 0.080000
train loss: 0.004353
val loss: 7.453687
(Epoch 6 / 10) val_acc: 0.080000
train loss: 0.002912
val loss: 7.242096
(Epoch 7 / 10) val_acc: 0.100000
train loss: 0.001838
val loss: 7.027784
(Epoch 8 / 10) val_acc: 0.100000
train loss: 0.001137
val loss: 6.822507
(Epoch 9 / 10) val_acc: 0.100000
train loss: 0.000705
val loss: 6.627312
(Epoch 10 / 10) val_acc: 0.120000


In [32]:
y_pred = model(X_train)
loss = loss_fn(last_output, y_train)
print (loss)
model

NameError: name 'last_output' is not defined